In [13]:
import ncd
from ncd import shape

## Dot-Product Attention
Using *Neural Circuit Diagrams* we can represent dot-product attention by;

<img src="Graphics/attention.png" width="700">

Each vertical section of a diagram corresponds to a shape. Columns either represent the data type we are working with, or an operation between data types. Solid lines represent axes and dashed lines separating terms represent Cartesian products. Placing a solid line adjacent to an operation, without separation, lifts it. Wirings represent Einstein operations, which includes linear contractions and rearrangements.

In [14]:
def display_columns(target: ncd.Shape, name = None):
    columns = '\n'.join([
        f'Domain:   {target.dom}',
        *(f'Morphism: {x}\n' + 
          f'Object:   {x.cod}' 
        for x in ncd.Composed.get_content(target))
    ])
    if name:
        columns = name + '\n' + columns
    return columns

In [15]:
from ncd import Duplicate
from ncd.nn import Einops, Linear, Addition, SoftMax
from itertools import starmap

L = Linear

x = shape('x')
m = shape('m^')
# Copying is implicit
linears = x >> (m @ (L('q') + L('k') + L('v')) @ 'k^')
# The axis names for einops are simply used as tags, which are attached
# to configurations.
ein1 = Einops('y k, x k -> y x')
softmax = x >> SoftMax()
ein2 = Einops('y x, x k -> y k')
linOut = L('o') @ m

# Note, the printout will contain tagged axes. These are configured
# upon composition.
section_names = ['Linears', 'Einops', 'SoftMax', 'Einops', 'Linear']
sections = [linears, ein1, softmax, ein2, linOut]
print("Printout of Individual Sections;")
print('\n'.join(starmap(display_columns, zip(sections, section_names))))

Printout of Individual Sections;
Linears
Domain:   [x m→]
Morphism: [x m→Δ3]
Object:   ([x m→], [x m→], [x m→])
Morphism: ([x→Lq], [x→Lk], [x→Lv])
Object:   ([x k→], [x k→], [x k→])
Einops
Domain:   ([y=y.3B k=k.1B→], [x=x.14 k=k.1B→])
Morphism: y k, x k -> y x
Object:   [y=y.3B x=x.14→]
SoftMax
Domain:   [x *=*.85→]
Morphism: [x→◁]
Object:   [x *=*.85→]
Einops
Domain:   ([y=y.88 x=x.77→], [x=x.77 k=k.E1→])
Morphism: y x, x k -> y k
Object:   [y=y.88 k=k.E1→]
Linear
Domain:   *=*.5E
Morphism: Lo
Object:   [m→]


In [16]:
attention = linears @ (ein1 @ softmax + '*') @ ein2 @ linOut
print("\nPrintout of Composed Expression;")
print(display_columns(attention))


Printout of Composed Expression;
Domain:   [x m→]
Morphism: [x m→Δ3]
Object:   ([x m→], [x m→], [x m→])
Morphism: ([x→Lq], [x→Lk], [x→Lv])
Object:   ([x k→], [x k→], [x k→])
Morphism: (y k, x k -> y x, [x k→])
Object:   ([x x→], [x k→])
Morphism: ([x→◁], [x k→])
Object:   ([x x→], [x k→])
Morphism: y x, x k -> y k
Object:   [x k→]
Morphism: Lo
Object:   [m→]


In [17]:
# We can use the marches package to disassemble an algebraic expression into
#   a graph, and to then compile it into code. Currently, PyTorch is supported.
import ncd.marches

# We use a functor which remaps objects to make them configurable.
make_configurable = ncd.DictFunctor({shape(a): ncd.Conf(a) for
    a in ['x', 'm', 'k']})

# See if our functor worked ie succesfully mapped set objects to
# configurable objects.
print(display_columns(make_configurable(attention)))

Domain:   [x=x.60 m=m.88→]
Morphism: [x=x.60 m=m.88→Δ3]
Object:   ([x=x.60 m=m.88→], [x=x.60 m=m.88→], [x=x.60 m=m.88→])
Morphism: ([x=x.60→Lq], [x=x.60→Lk], [x=x.60→Lv])
Object:   ([x=x.60 k=k.20→], [x=x.60 k=k.20→], [x=x.60 k=k.20→])
Morphism: (y k, x k -> y x, [x=x.60 k=k.20→])
Object:   ([x=x.60 x=x.60→], [x=x.60 k=k.20→])
Morphism: ([x=x.60→◁], [x=x.60 k=k.20→])
Object:   ([x=x.60 x=x.60→], [x=x.60 k=k.20→])
Morphism: y x, x k -> y k
Object:   [x=x.60 k=k.20→]
Morphism: Lo
Object:   [m=m.88→]


In [18]:
# It did! So we can compile it, with a correct __init__ function.
# 'Multilinear' is found in ncd.torch_utilities
print(ncd.marches.to_torch(make_configurable(attention), "Attention"))

class Attention(nn.Module):
    def __init__(self, k, m, x):
        self.Lq = Multilinear((m),(k))
        self.Lk = Multilinear((m),(k))
        self.Lv = Multilinear((m),(k))
        self.Lo = Multilinear((x, k),(m))
    def forward(self, a):
        a, b, c = a, a, a
        a = Lq(a)
        b = Lk(b)
        c = Lv(c)
        a = einops.einsum(a, b, "y k, x k -> y x")
        a = torch.softmax(a, dim=-1)
        a = einops.einsum(a, c, "y x, x k -> y k")
        a = Lo(a)
        return a


## Multi-Head Dot Product Attention
We represent the more intricate multi-head dot-product attention by;

<img src="Graphics/multihead.png" width="700">

Compared to single-head dot product attention above, this diagram has an additional $h$ axis. The linear layers output data of size ``k h``, there is additional wiring for the Einops, and the SoftMax is lifted below. We can implement these changes using our algebraic tools.

In [19]:
# We will first construct Multi-Head Attention without configurables
# (as above), then show a construction using configurables all the
# way through.

from ncd import Duplicate, shape
from ncd.nn import Einops, Linear, Addition, SoftMax

L = Linear
# The lower-most axis represents R^m
m = shape('m^')
# x represents the number of tokens. It is an integer.
x = shape('x')
# Addition assembles shapes into tuples.
linears = x >> (m @ (L('q') + L('k') + L('v')) @ 'k h')
# The first Einops contracts the k dimension.
# We tuple it with a generic shape '*' to leave the values
# tuple segment unchanged.
einops = Einops('q k h, x k h -> q x h')
softmax = x >> SoftMax() << 'h'
einops2 = Einops('q x h, x k h -> q k h')
linout = shape('k h') @ L('o') @ 'm'

multihead = linears @ (einops @ softmax + '*') @ einops2 @ linout
print(display_columns(multihead))

Domain:   [x m→]
Morphism: [x m→Δ3]
Object:   ([x m→], [x m→], [x m→])
Morphism: ([x→Lq], [x→Lk], [x→Lv])
Object:   ([x k h→], [x k h→], [x k h→])
Morphism: (q k h, x k h -> q x h, [x k h→])
Object:   ([x x h→], [x k h→])
Morphism: ([x→◁←h], [x k h→])
Object:   ([x x h→], [x k h→])
Morphism: q x h, x k h -> q k h
Object:   [x k h→]
Morphism: [x→Lo]
Object:   [x→m]


In [20]:
make_configurable = ncd.DictFunctor({shape(a): ncd.Conf(a) for
    a in ['x', 'm', 'k', 'h']})
import ncd.marches
print(ncd.marches.to_torch(make_configurable(multihead), "MultiHeadAttention"))

class MultiHeadAttention(nn.Module):
    def __init__(self, h, k, m, x):
        self.Lq = Multilinear((m),(k, h))
        self.Lk = Multilinear((m),(k, h))
        self.Lv = Multilinear((m),(k, h))
        self.Lo = Multilinear((k, h),m)
    def forward(self, a):
        a, b, c = a, a, a
        a = Lq(a)
        b = Lk(b)
        c = Lv(c)
        a = einops.einsum(a, b, "q k h, x k h -> q x h")
        a = torch.softmax(a, dim=-2)
        a = einops.einsum(a, c, "q x h, x k h -> q k h")
        a = Lo(a)
        return a


In [21]:
# We can also use configurations all the way through, and analyse the individual
# sections. This shows how the expressions we use correspond with the diagram,
# and how the composition process automatically aligns the configurable axes.

from ncd import Duplicate, shape
from ncd.nn import Einops, Linear, Addition, SoftMax

# Multi-Headed Attention defined symbolically.
# We piece together individual sections.
# Upon composition, axes sizes are aligned!
L = Linear
m = shape('*m^')
x = shape('*x')

linears = x >> (m @ (L('q') + L('k') + L('v')) @ '*k *h')
einops = (Einops('q k h, x k h -> q x h') + '*')
softs = ((x >> SoftMax() << '*') + '*') @ Einops('q x h, x k h -> q k h')
linout = (shape('*k *h') @ L('o') @ m)

# Note, the printout will contain tagged axes. These are configured
# upon composition.
section_names = ['Linears', 'Einops', 'SoftMax + Einops', 'Linear']
sections = [linears, einops, softs, linout]
print("Printout of Individual Sections;")
print('\n'.join(starmap(display_columns, zip(sections, section_names))))

Printout of Individual Sections;
Linears
Domain:   [x=x.4C m=m.A5→]
Morphism: [x=x.4C m=m.A5→Δ3]
Object:   ([x=x.4C m=m.A5→], [x=x.4C m=m.A5→], [x=x.4C m=m.A5→])
Morphism: ([x=x.4C→Lq], [x=x.4C→Lk], [x=x.4C→Lv])
Object:   ([x=x.4C k=k.2D h=h.36→], [x=x.4C k=k.2D h=h.36→], [x=x.4C k=k.2D h=h.36→])
Einops
Domain:   ([q=q.43 k=k.63 h=h.6C→], [x=x.E0 k=k.63 h=h.6C→], =.A8)
Morphism: (q k h, x k h -> q x h, =.A8)
Object:   ([q=q.43 x=x.E0 h=h.6C→], =.A8)
SoftMax + Einops
Domain:   ([x=x.4C *=x.DF =h.51→], [x=x.DF k=k.D7 h=h.51→])
Morphism: ([x=x.4C→◁←=h.51], [x=x.DF k=k.D7 h=h.51→])
Object:   ([x=x.4C *=x.DF =h.51→], [x=x.DF k=k.D7 h=h.51→])
Morphism: q x h, x k h -> q k h
Object:   [q=x.4C k=k.D7 h=h.51→]
Linear
Domain:   [k=k.F3 h=h.15→]
Morphism: Lo
Object:   [m=m.A5→]


In [22]:
multihead = linears @ einops @ softs @ (shape('*k *h') @ L('o') @ m)

print(display_columns(multihead))

Domain:   [x=x.E0 m=m.A5→]
Morphism: [x=x.E0 m=m.A5→Δ3]
Object:   ([x=x.E0 m=m.A5→], [x=x.E0 m=m.A5→], [x=x.E0 m=m.A5→])
Morphism: ([x=x.E0→Lq], [x=x.E0→Lk], [x=x.E0→Lv])
Object:   ([x=x.E0 k=k.D7 h=h.6C→], [x=x.E0 k=k.D7 h=h.6C→], [x=x.E0 k=k.D7 h=h.6C→])
Morphism: (q k h, x k h -> q x h, [x=x.E0 k=k.D7 h=h.6C→])
Object:   ([q=x.E0 x=x.E0 h=h.6C→], [x=x.E0 k=k.D7 h=h.6C→])
Morphism: ([x=x.E0→◁←=h.6C], [x=x.E0 k=k.D7 h=h.6C→])
Object:   ([x=x.E0 *=x.E0 =h.6C→], [x=x.E0 k=k.D7 h=h.6C→])
Morphism: q x h, x k h -> q k h
Object:   [q=x.E0 k=k.D7 h=h.6C→]
Morphism: [q=x.E0→Lo]
Object:   [q=x.E0 m=m.A5→]


In [23]:
# We can use the "GetConfig" functor to accumulate the
# unassigned variables in its internal state. This allows
# us to quickly generate configuration parameters from an
# expression.
config = ncd.GetConfig()
config(multihead)
print(config.configs)

{x.E0, h.6C, m.A5, k.D7}


In [24]:
# Marches is a package for compiling code.
# Here, it generate code for multi-headed attention.
import ncd.marches

print(ncd.marches.to_torch(multihead, "MultiHeadAttention"))

class MultiHeadAttention(nn.Module):
    def __init__(self, x, h, m, k):
        self.Lq = Multilinear((m),(k, h))
        self.Lk = Multilinear((m),(k, h))
        self.Lv = Multilinear((m),(k, h))
        self.Lo = Multilinear((k, h),(m))
    def forward(self, a):
        a, b, c = a, a, a
        a = Lq(a)
        b = Lk(b)
        c = Lv(c)
        a = einops.einsum(a, b, "q k h, x k h -> q x h")
        a = torch.softmax(a, dim=-2)
        a = einops.einsum(a, c, "q x h, x k h -> q k h")
        a = Lo(a)
        return a
